-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Writing to Delta Tables
Delta Lake tables provide ACID compliant updates to tables backed by data files in cloud object storage.

In this notebook, we'll explore SQL syntax to process updates with Delta Lake. While many operations are standard SQL, slight variations exist to accommodate Spark and Delta Lake execution.

## Learning Objectives
By the end of this lesson, you should be able to:
- Overwrite data tables using **`INSERT OVERWRITE`**
- Append to a table using **`INSERT INTO`**
- Append, update, and delete from a table using **`MERGE INTO`**
- Ingest data incrementally into tables using **`COPY INTO`**

## Run Setup

The setup script will create the data and declare necessary values for the rest of this notebook to execute.

In [0]:
%run ../Includes/Classroom-Setup-4.4

Python interpreter will be restarted.
Python interpreter will be restarted.



Creating the database "dbacademy_chiraggoel_kpmg_com_dewd_4_4"
Skipping install to "dbfs:/user/chiraggoel@kpmg.com/dbacademy/dewd/source/eltwss", dataset already exists

Cloning the sales table from dbfs:/user/chiraggoel@kpmg.com/dbacademy/dewd/source/eltwss/delta/sales_hist...(8 seconds / 10,510 records)
Cloning the users table from dbfs:/user/chiraggoel@kpmg.com/dbacademy/dewd/source/eltwss/delta/users_hist...(6 seconds / 251,501 records)
Cloning the events table from dbfs:/user/chiraggoel@kpmg.com/dbacademy/dewd/source/eltwss/delta/events_hist...(8 seconds / 485,696 records)
Cloning the users_update table from dbfs:/user/chiraggoel@kpmg.com/dbacademy/dewd/source/eltwss/delta/users_update...(7 seconds / 917 records)
Cloning the events_update table from dbfs:/user/chiraggoel@kpmg.com/dbacademy/dewd/source/eltwss/delta/events_update...(6 seconds / 1,927 records)

Predefined Paths:
  DA.paths.working_dir: dbfs:/user/chiraggoel@kpmg.com/dbacademy/dewd/4.4
  DA.paths.user_db:     dbfs:/u

## Complete Overwrites

We can use overwrites to atomically replace all of the data in a table. There are multiple benefits to overwriting tables instead of deleting and recreating tables:
- Overwriting a table is much faster because it doesn’t need to list the directory recursively or delete any files.
- The old version of the table still exists; can easily retrieve the old data using Time Travel.
- It’s an atomic operation. Concurrent queries can still read the table while you are deleting the table.
- Due to ACID transaction guarantees, if overwriting the table fails, the table will be in its previous state.

Spark SQL provides two easy methods to accomplish complete overwrites.

Some students may have noticed previous lesson on CTAS statements actually used CRAS statements (to avoid potential errors if a cell was run multiple times).

**`CREATE OR REPLACE TABLE`** (CRAS) statements fully replace the contents of a table each time they execute.

In [0]:
%sql
CREATE OR REPLACE TABLE events AS
SELECT * FROM parquet.`${da.paths.datasets}/raw/events-historical`

num_affected_rows,num_inserted_rows


Reviewing the table history shows a previous version of this table was replaced.

In [0]:
%sql
DESCRIBE HISTORY events

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2022-05-22T13:06:47.000+0000,6138111496777303,chiraggoel@kpmg.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2841292000076347),0522-095621-z1upwij9,0,WriteSerializable,false,"Map(numFiles -> 4, numOutputRows -> 485696, numOutputBytes -> 15283922)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-05-22T13:06:01.000+0000,6138111496777303,chiraggoel@kpmg.com,CLONE,"Map(source -> delta.`dbfs:/user/chiraggoel@kpmg.com/dbacademy/dewd/source/eltwss/delta/events_hist`, sourceVersion -> 1, isShallow -> true)",null,List(2841292000076347),0522-095621-z1upwij9,-1,Serializable,false,"Map(removedFilesSize -> 0, numRemovedFiles -> 0, sourceTableSize -> 14998176, numCopiedFiles -> 0, copiedFilesSize -> 0, sourceNumOfFiles -> 1)",null,Databricks-Runtime/10.4.x-scala2.12


**`INSERT OVERWRITE`** provides a nearly identical outcome as above: data in the target table will be replaced by data from the query. 

**`INSERT OVERWRITE`**:

- Can only overwrite an existing table, not create a new one like our CRAS statement
- Can overwrite only with new records that match the current table schema -- and thus can be a "safer" technique for overwriting an existing table without disrupting downstream consumers
- Can overwrite individual partitions

In [0]:
%sql
INSERT OVERWRITE sales
SELECT * FROM parquet.`${da.paths.datasets}/raw/sales-historical/`

num_affected_rows,num_inserted_rows
10510,10510


Note that different metrics are displayed than a CRAS statement; the table history also records the operation differently.

In [0]:
%sql
DESCRIBE HISTORY sales

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2022-05-22T13:07:03.000+0000,6138111496777303,chiraggoel@kpmg.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(2841292000076347),0522-095621-z1upwij9,0,WriteSerializable,false,"Map(numFiles -> 4, numOutputRows -> 10510, numOutputBytes -> 353497)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-05-22T13:05:48.000+0000,6138111496777303,chiraggoel@kpmg.com,CLONE,"Map(source -> delta.`dbfs:/user/chiraggoel@kpmg.com/dbacademy/dewd/source/eltwss/delta/sales_hist`, sourceVersion -> 1, isShallow -> true)",null,List(2841292000076347),0522-095621-z1upwij9,-1,Serializable,false,"Map(removedFilesSize -> 0, numRemovedFiles -> 0, sourceTableSize -> 334175, numCopiedFiles -> 0, copiedFilesSize -> 0, sourceNumOfFiles -> 1)",null,Databricks-Runtime/10.4.x-scala2.12


A primary difference here has to do with how Delta Lake enforces schema on write.

Whereas a CRAS statement will allow us to completely redefine the contents of our target table, **`INSERT OVERWRITE`** will fail if we try to change our schema (unless we provide optional settings). 

Uncomment and run the cell below to generated an expected error message.

In [0]:
%sql
INSERT OVERWRITE sales
SELECT *, current_timestamp() FROM parquet.`${da.paths.datasets}/raw/sales-historical`

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: c72fe65c-a9de-4cf0-816a-a92c5d1a0b96).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- order_id: long (nullable = true)
-- email: string (nullable = true)
-- transaction_timestamp: long (nullable = true)
-- total_item_quantity: long (nullable = true)
-- purchase_revenue_in_usd: double (nullable = true)
-- unique_items: long (nullable = true)
-- items: array (nullable = true)
-- element: struct (containsNull = true)
 |-- coupon: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- item_revenue_in_usd: double (nullable = true)
 |-- price_in_usd: double (nullable = true)
 |-- quantity: long (nullable = true)


Data schema:
root
-- order_id: long (nullable = true)
-- email: string (nullable = true)
-- transaction_timestamp: long (nullable = true)
-- total_item_quantity: long (nullable = true)
-- purchase_revenue_in_usd: double (nullable = true)
-- unique_items: long (nullable = true)
-- items: array (nullable = true)
-- element: struct (containsNull = true)
 |-- coupon: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- item_revenue_in_usd: double (nullable = true)
 |-- price_in_usd: double (nullable = true)
 |-- quantity: long (nullable = true)
-- current_timestamp(): timestamp (nullable = true)

 
To overwrite your schema or change partitioning, please set:
'.option("overwriteSchema", "true")'.

Note that the schema can't be overwritten when using
'replaceWhere'.
 
	at com.databricks.sql.transaction.tahoe.MetadataMismatchErrorBuilder.finalizeAndThrow(DeltaErrors.scala:1962)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation.updateMetadata(ImplicitMetadataOperation.scala:131)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation.updateMetadata$(ImplicitMetadataOperation.scala:55)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.updateMetadata(WriteIntoDelta.scala:63)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.write(WriteIntoDelta.scala:143)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.$anonfun$run$2(WriteIntoDelta.scala:92)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.$anonfun$run$2$adapted(WriteIntoDelta.scala:84)
	at com.databricks.sql.transaction.tahoe.DeltaLog.withNewTransaction(DeltaLog.scala:235)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.$anonfun$run$1(WriteIntoDelta.scala:84)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:1584)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDelta.run(WriteIntoDelta.scala:83)
	at com.databricks.sql.transaction.tahoe.catalog.WriteIntoDeltaBuilder$$anon$1$$anon$2.insert(DeltaTableV2.scala:291)
	at org.apache.spark.sql.execution.datasources.v2.SupportsV1Write.writeWithV1(V1FallbackWriters.scala:84)
	at org.apache.spark.sql.execution.datasources.v2.SupportsV1Write.writeWithV1$(V1FallbackWriters.scala:82)
	at org.apache.spark.sql.execution.datasources.v2.OverwriteByExpressionExecV1.writeWithV1(V1FallbackWriters.scala:53)
	at org.apache.spark.sql.execution.datasources.v2.V1FallbackWriters.run(V1FallbackWriters.scala:70)
	at org.apache.spark.sql.execution.datasources.v2.V1FallbackWriters.run$(V1FallbackWriters.scala:69)
	at org.apache.spark.sql.execution.datasources.v2.OverwriteByExpressionExecV1.run(V1FallbackWriters.scala:53)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:43)
	at org.apache.spark.sql.exe

## Append Rows

We can use **`INSERT INTO`** to atomically append new rows to an existing Delta table. This allows for incremental updates to existing tables, which is much more efficient than overwriting each time.

Append new sale records to the **`sales`** table using **`INSERT INTO`**.

In [0]:
%sql
INSERT INTO sales
SELECT * FROM parquet.`${da.paths.datasets}/raw/sales-30m`

num_affected_rows,num_inserted_rows
29,29


Note that **`INSERT INTO`** does not have any built-in guarantees to prevent inserting the same records multiple times. Re-executing the above cell would write the same records to the target table, resulting in duplicate records.

## Merge Updates

You can upsert data from a source table, view, or DataFrame into a target Delta table using the **`MERGE`** SQL operation. Delta Lake supports inserts, updates and deletes in **`MERGE`**, and supports extended syntax beyond the SQL standards to facilitate advanced use cases.

<strong><code>
MERGE INTO target a<br/>
USING source b<br/>
ON {merge_condition}<br/>
WHEN MATCHED THEN {matched_action}<br/>
WHEN NOT MATCHED THEN {not_matched_action}<br/>
</code></strong>

We will use the **`MERGE`** operation to update historic users data with updated emails and new users.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW users_update AS 
SELECT *, current_timestamp() AS updated 
FROM parquet.`${da.paths.datasets}/raw/users-30m`

The main benefits of **`MERGE`**:
* updates, inserts, and deletes are completed as a single transaction
* multiple conditionals can be added in addition to matching fields
* provides extensive options for implementing custom logic

Below, we'll only update records if the current row has a **`NULL`** email and the new row does not. 

All unmatched records from the new batch will be inserted.

In [0]:
%sql
MERGE INTO users a
USING users_update b
ON a.user_id = b.user_id
WHEN MATCHED AND a.email IS NULL AND b.email IS NOT NULL THEN
  UPDATE SET email = b.email, updated = b.updated
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
983,72,0,911


Note that we explicitly specify the behavior of this function for both the **`MATCHED`** and **`NOT MATCHED`** conditions; the example demonstrated here is just an example of logic that can be applied, rather than indicative of all **`MERGE`** behavior.

## Insert-Only Merge for Deduplication

A common ETL use case is to collect logs or other every-appending datasets into a Delta table through a series of append operations. 

Many source systems can generate duplicate records. With merge, you can avoid inserting the duplicate records by performing an insert-only merge.

This optimized command uses the same **`MERGE`** syntax but only provided a **`WHEN NOT MATCHED`** clause.

Below, we use this to confirm that records with the same **`user_id`** and **`event_timestamp`** aren't already in the **`events`** table.

In [0]:
%sql
MERGE INTO events a
USING events_update b
ON a.user_id = b.user_id AND a.event_timestamp = b.event_timestamp
WHEN NOT MATCHED AND b.traffic_source = 'email' THEN 
  INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
128,0,0,128


## Load Incrementally

**`COPY INTO`** provides SQL engineers an idempotent option to incrementally ingest data from external systems.

Note that this operation does have some expectations:
- Data schema should be consistent
- Duplicate records should try to be excluded or handled downstream

This operation is potentially much cheaper than full table scans for data that grows predictably.

While here we'll show simple execution on a static directory, the real value is in multiple executions over time picking up new files in the source automatically.

In [0]:
%sql
COPY INTO sales
FROM "${da.paths.datasets}/raw/sales-30m"
FILEFORMAT = PARQUET

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: java.lang.UnsupportedOperationException: com.databricks.backend.daemon.data.client.DBFSV1.createAtomicIfAbsent(path: Path)
	at com.databricks.tahoe.store.EnhancedDatabricksFileSystemV1.createAtomicIfAbsent(EnhancedFileSystem.scala:324)
	at com.databricks.spark.sql.streaming.AWSCheckpointFileManager.createAtomicIfAbsent(DatabricksCheckpointFileManager.scala:159)
	at com.databricks.spark.sql.streaming.DatabricksCheckpointFileManager.createAtomicIfAbsent(DatabricksCheckpointFileManager.scala:60)
	at org.apache.spark.sql.execution.streaming.state.RocksDBFileManager.zipToDfsFile(RocksDBFileManager.scala:510)
	at org.apache.spark.sql.execution.streaming.state.RocksDBFileManager.saveCheckpointToDfs(RocksDBFileManager.scala:189)
	at com.databricks.sql.rocksdb.CloudRocksDB.$anonfun$open$5(CloudRocksDB.scala:467)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.timeTakenMs(Utils.scala:680)
	at com.databricks.sql.rocksdb.CloudRocksDB.timeTakenMs(CloudRocksDB.scala:543)
	at com.databricks.sql.rocksdb.CloudRocksDB.$anonfun$open$2(CloudRocksDB.scala:455)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:444)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:28)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:26)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:205)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:204)
	at com.databricks.spark.util.PublicDBLogging.withAttributionContext(DatabricksSparkUsageLogger.scala:20)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:240)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:225)
	at com.databricks.spark.util.PublicDBLogging.withAttributionTags(DatabricksSparkUsageLogger.scala:20)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:419)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:339)
	at com.databricks.spark.util.PublicDBLogging.recordOperationWithResultTags(DatabricksSparkUsageLogger.scala:20)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:302)
	at com.databricks.spark.util.PublicDBLogging.recordOperation(DatabricksSparkUsageLogger.scala:20)
	at com.databricks.spark.util.PublicDBLogging.recordOperation0(DatabricksSparkUsageLogger.scala:57)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:137)
	at com.databricks.spark.util.UsageLogger.recordOperation(UsageLogger.scala:73)
	at com.databricks.spark.util.UsageLogger.recordOperation$(UsageLogger.scala:60)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:98)
	at com.databricks.spark.util.UsageLogging.recordOperation(UsageLogger.scala:431)
	at com.databricks.spark.util.UsageLogging.recordOperation$(UsageLogger.scala:410)
	at com.databricks.sql.rocksdb.CloudRocksDB.recordOperation(CloudRocksDB.scala:55)
	at com.databricks.sql.rocksdb.CloudRocksDB.recordRocksDBOperation(CloudRocksDB.scala:562)
	at com.databricks.sql.rocksdb.CloudRocksDB.$anonfun$open$1(CloudRocksDB.scala:447)
	at com.databricks.backend.daemon.driver.ProgressReporter$.withStatusCode(ProgressReporter.scala:377)
	at com.databri

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
%python
DA.cleanup()

Dropping the database "dbacademy_chiraggoel_kpmg_com_dewd_4_4"
Removing the working directory "dbfs:/user/chiraggoel@kpmg.com/dbacademy/dewd/4.4"


-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>